### Intro

So my first notebook, learning through doing.
Please, point out my mistakes and stupidity in comments, i do apprecciate it.

In [ ]:
import numpy as np # linear algebra
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('../input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')
df.head()
df.drop(['id'], axis=1, inplace = True)

### Visualisation

In [ ]:
plt.rcParams['figure.dpi'] = 100
fig, (ax4, ax2, ax3) = plt.subplots(1, 3, figsize=(12, 4))
size = 0.4


cmap = plt.get_cmap("tab20c")
outer_colors = cmap(np.arange(3)*4)


#Stroke married
ax4.pie(df[df['ever_married']=='Yes']['stroke'].value_counts(), radius=1, colors=outer_colors,
       wedgeprops=dict(width=size, edgecolor='w'), autopct='%1.1f%%', pctdistance=0.75)
ax4.pie(df[df['ever_married']=='No']['stroke'].value_counts(), radius=1-size, colors=outer_colors,
       wedgeprops=dict(width=size, edgecolor='w'), autopct='%1.1f%%', pctdistance=0.75)
ax4.set(aspect="equal", title='Married\nOuter - 1, Inner -0')




#Overall
ax2.text(0, 0, f'{df["stroke"].count()} cases', color='black', va="center", ha="center")
ax2.pie(df['stroke'].value_counts(), radius=1, colors=outer_colors,
       wedgeprops=dict(width=size, edgecolor='w'), autopct='%1.1f%%', pctdistance=0.75)
ax2.set(aspect="equal", title='Stroke overall')



#Hypertension
ax3.pie(df[df['hypertension']==1]['stroke'].value_counts(), radius=1, colors=outer_colors,
       wedgeprops=dict(width=size, edgecolor='w'), autopct='%1.1f%%', pctdistance=0.75)
ax3.pie(df[df['hypertension']==0]['stroke'].value_counts(), radius=1-size, colors=outer_colors,
       wedgeprops=dict(width=size, edgecolor='w'), autopct='%1.1f%%', pctdistance=0.75)
ax3.set(aspect="equal", title='Hypertension\nOuter - 1, Inner -0')


fig.legend(['Stroke', 'No stroke'], loc='upper right')


plt.show()

We've got a very imbalanced data. Let's make a model.

### Missing values

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

Options:
1. Drop NaN values 
2. Drop whole column

BMI feature doesn't seem to make some influence on stroke possibility. I decided to drop whole column.

In [ ]:
#df.drop('bmi', axis=1, inplace=True)
df['bmi'].fillna(df['bmi'].mean(), inplace=True)

### Categorical values

In [ ]:
#Separate Data into numerical and categorical columns
cat_cols = df.select_dtypes(['object', 'int']).columns
num_cols = df.select_dtypes(['float']).columns

print(f'Categorical columns: {cat_cols.values}. \n Numerical columns: {num_cols.values}.')

In [ ]:
#Gender uniques
df['gender'].value_counts()

In [ ]:
#Just one value, dropped
df.drop(df[df['gender'] == 'Other'].index, axis=0, inplace=True)

In [ ]:
#Map binary categories to numerical
df['Residence_type'] = df['Residence_type'].map({'Rural': 0, 'Urban': 1})
df['ever_married'] = df['ever_married'].map({'No': 0, 'Yes': 1})
df['gender'] = df['gender'].map({'Male': 0, 'Female': 1})

2 multinominal categories left:
* smoking_status - label encoding (Unknown = never smoked)
* work_type - OHE

In [ ]:
df['smoking_status'].unique()

In [ ]:
df['smoking_status'] = df['smoking_status'].map({'Unknown': 0, 'never smoked': 0, 'formerly smoked': 1, 'smokes': 2})

In [ ]:
ohe = OneHotEncoder()

#I used index=df.index, cuz I deleted row before and OHE indexing dont match with df.

df = pd.concat([df, pd.DataFrame(ohe.fit_transform(df[['work_type']]).toarray(), columns=ohe.categories_[0], index=df.index)], 
          axis=1)
df.drop('work_type', axis=1, inplace=True)

In [ ]:
df.head()

> ### Modelling

In [ ]:
# Take target column away and split X
y = df['stroke']
X = df.drop('stroke', axis=1)

In [ ]:
#Scaling
scaler = StandardScaler()
X = scaler.fit_transform(X)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score, auc, precision_score, recall_score
import datetime

Simple Models with default hyper-parameters

In [ ]:
models = [
    {'name': 'LogisticRegression', 'estimator': LogisticRegression(random_state=42)},
    {'name': 'GradientBoostingClassifier', 'estimator': GradientBoostingClassifier(random_state=42)},
    {'name': 'AdaBoostClassifier', 'estimator': AdaBoostClassifier(random_state=42)},
    {'name': 'BaggingClassifier', 'estimator': BaggingClassifier(random_state=42)},
    {'name': 'XGBClassifier', 'estimator': XGBClassifier(verbosity=0, random_state=42)},
    {'name': 'DecisionTreeClassifier', 'estimator': DecisionTreeClassifier(random_state=42)},
    
]


In [ ]:
print('Name           \t Time          \t Recall      ')
print('--------------------------------------------------------')
for model in models:
    time_a = datetime.datetime.now()
    
    model['estimator'].fit(X_train, y_train)
    model['preds'] =  model['estimator'].predict(X_test)
    model['recall'] = recall_score(y_test, model['preds'], average='binary', pos_label=1)
    
    time_b = datetime.datetime.now()
    print(f"{model['name']}\t {time_b-time_a}\t {round(model['recall'],4)}\t")

Best estimator gives us 0.1348 recall. Test target proportion:

In [ ]:
y_test.sum() / y_test.count()

# WELL.....

<img src= "https://i.kym-cdn.com/entries/icons/original/000/028/021/work.jpg">

Data is heavy imbalanced, Ill try to improve results in next kernel versions. Stay in touch.

But I got one insight:

 *Dont get married if you dont want a stroke (6ix-time risk increased... yeah, let this dataset represenet the whole population*
 

